# Model Development Programming Exercise

In [928]:
# import all the modules here
import pandas as pd
import numpy as np
import os
from datetime import timedelta

## Task 1: Import data.

In [929]:
contracts = pd.read_csv('./contracts/Contracts.csv')
plantParamenters = pd.read_csv('./plantParameters/Plant_Parameters.csv')

In [930]:
contracts.head()

,ContractName,DealType,StartDate,EndDate,Volume,Granularity,StrikePrice,Premium,PriceName
0,S1,Swap,2017-01-01,2017-03-31,20000,Daily,3.0,NaN,Henry Hub
1,S2,Swap,2017-01-01,2018-12-31,1000,Hourly,21.0,NaN,HB_NORTH
2,O1,European option,2017-06-01,2017-07-15,10000,Daily,2.9,0.1,GDA_TETSTX
3,O2,European option,2018-10-01,2019-09-30,1000,Hourly,31.5,3.0,HB_HOUSTON


In [931]:
os.listdir('./powerPrices/')

['ERCOT_DA_Prices_2017.csv',
 'ERCOT_DA_Prices_2016.csv',
 'ERCOT_DA_Prices_2018.csv',
 'ERCOT_DA_Prices_2019.csv']

In [932]:
plantParamenters.head()

,PlantName,Year,Month,PowerPriceName,FuelPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost
0,Alpha,2017,1,HB_HOUSTON,Henry Hub,100,9.2,3,0.1,10000
1,Alpha,2017,2,HB_HOUSTON,Henry Hub,98,9.4,3,0.1,10000
2,Alpha,2017,3,HB_HOUSTON,Henry Hub,96,9.6,3,0.1,10000
3,Alpha,2017,4,HB_HOUSTON,Henry Hub,94,9.8,3,0.1,10000
4,Alpha,2017,5,HB_HOUSTON,Henry Hub,92,10.0,3,0.1,10000


In [933]:
fuelP = pd.DataFrame()
for x in os.listdir('./fuelPrices/'):
    df_tmp = pd.read_csv('./fuelPrices/'+x)
    fuelP = pd.concat([fuelP,df_tmp],ignore_index=True)
fuelP.Date = fuelP['Date'].apply(pd.Timestamp)
fuelP

,Variable,Date,Price
0,GDA_TETSTX,2019-05-01,2.490
1,GDA_TETSTX,2019-04-30,2.420
2,GDA_TETSTX,2019-04-29,2.445
3,GDA_TETSTX,2019-04-28,2.445
4,GDA_TETSTX,2019-04-27,2.445
...,...,...,...
3677,Henry Hub,2020-09-26,1.895
3678,Henry Hub,2020-09-27,1.895
3679,Henry Hub,2020-09-28,1.895
3680,Henry Hub,2020-09-29,1.835


In [934]:
powerP = pd.DataFrame()
for x in os.listdir('./powerPrices/'):
    df_tmp = pd.read_csv('./powerPrices/'+x)
    powerP = pd.concat([powerP,df_tmp],ignore_index=True)
powerP.Date = powerP['Date'].apply(pd.Timestamp)
powerP

,Date,SettlementPoint,Price
0,2017-01-01 00:00:00,HB_BUSAVG,23.23
1,2017-01-01 00:00:00,HB_HOUSTON,23.23
2,2017-01-01 00:00:00,HB_HUBAVG,23.23
3,2017-01-01 00:00:00,HB_NORTH,23.23
4,2017-01-01 00:00:00,HB_SOUTH,23.23
...,...,...,...
497315,2019-12-31 23:00:00,LZ_LCRA,14.53
497316,2019-12-31 23:00:00,LZ_NORTH,14.48
497317,2019-12-31 23:00:00,LZ_RAYBN,14.48
497318,2019-12-31 23:00:00,LZ_SOUTH,14.70


--------------
# Power Price Statistics
## Task 2: Calculate basic descriptive statistics

In [935]:
powerStats = powerP.set_index('Date').to_period('M').groupby(['SettlementPoint','Date']).describe()
powerStats.columns = [x[1] for x in powerStats.columns]
powerStats.head()

count       mean       std   min      25%     50%  \
SettlementPoint Date                                                         
HB_BUSAVG       2016-01  744.0  19.209301  5.485210  5.12  16.3300  18.630   
                2016-02  696.0  15.288247  5.777172  1.79  12.9000  15.235   
                2016-03  743.0  16.755626  7.368497  1.22  12.4850  15.760   
                2016-04  720.0  18.289347  9.506143  1.18  12.8225  17.150   
                2016-05  744.0  18.159583  8.017221  3.03  12.5200  16.955   

                             75%    max  
SettlementPoint Date                     
HB_BUSAVG       2016-01  21.1200  54.69  
                2016-02  17.6525  50.98  
                2016-03  19.9050  64.31  
                2016-04  21.9000  93.09  
                2016-05  21.3700  73.61

## Task 3: Calculate volatility

In [936]:
def f(x):
    # handle negative price
    x = x + abs(np.min(x)) + 0.01
    return np.std(np.log(x[1:]/x[:-1]))
vol = powerP.set_index('Date').sort_index().to_period('M').groupby(['SettlementPoint','Date']).apply(f)
vol.columns = ['vol']
vol.head()

vol
SettlementPoint Date             
HB_BUSAVG       2016-01  0.131961
                2016-02  0.190387
                2016-03  0.191355
                2016-04  0.202196
                2016-05  0.141286

## Task 4: Write the results to file

In [937]:
powerStats[['mean','min','max','std']].join(vol,on=['SettlementPoint','Date']).to_csv('./output_files/MonthlyPowerPriceStatistics.csv')

---------------
# Contract Valuation
## Task 5: Expand the contracts across relevant time periods.

In [938]:
contracts.StartDate = contracts['StartDate'].apply(pd.Timestamp)
contracts.EndDate = contracts['EndDate'].apply(pd.Timestamp)
daily_c = contracts[contracts['Granularity']=='Daily'].reset_index(drop=True)
hourly_c = contracts[contracts['Granularity']=='Hourly'].reset_index(drop=True)

In [939]:
daily_c

,ContractName,DealType,StartDate,EndDate,Volume,Granularity,StrikePrice,Premium,PriceName
0,S1,Swap,2017-01-01,2017-03-31,20000,Daily,3.0,NaN,Henry Hub
1,O1,European option,2017-06-01,2017-07-15,10000,Daily,2.9,0.1,GDA_TETSTX


In [940]:
hourly_c

,ContractName,DealType,StartDate,EndDate,Volume,Granularity,StrikePrice,Premium,PriceName
0,S2,Swap,2017-01-01,2018-12-31,1000,Hourly,21.0,NaN,HB_NORTH
1,O2,European option,2018-10-01,2019-09-30,1000,Hourly,31.5,3.0,HB_HOUSTON


In [941]:
idx_new = list(daily_c['StartDate'])+list(daily_c['EndDate'])
time_idx = pd.DataFrame(idx_new,columns=['StartDate']).set_index('StartDate').asfreq(freq='1D')
tmp = time_idx.join(daily_c.set_index('StartDate'),on='StartDate')
daily_c = tmp.loc[(tmp.index<idx_new[2])| (tmp.index>=idx_new[1]) & (tmp.index<idx_new[3])].ffill()
daily_c.index = daily_c.index.rename('Date')
daily_c

,ContractName,DealType,EndDate,Volume,Granularity,StrikePrice,Premium,PriceName
Date,,,,,,,,
2017-01-01,S1,Swap,2017-03-31,20000.0,Daily,3.0,NaN,Henry Hub
2017-01-02,S1,Swap,2017-03-31,20000.0,Daily,3.0,NaN,Henry Hub
2017-01-03,S1,Swap,2017-03-31,20000.0,Daily,3.0,NaN,Henry Hub
2017-01-04,S1,Swap,2017-03-31,20000.0,Daily,3.0,NaN,Henry Hub
2017-01-05,S1,Swap,2017-03-31,20000.0,Daily,3.0,NaN,Henry Hub
...,...,...,...,...,...,...,...,...
2017-07-10,O1,European option,2017-07-15,10000.0,Daily,2.9,0.1,GDA_TETSTX
2017-07-11,O1,European option,2017-07-15,10000.0,Daily,2.9,0.1,GDA_TETSTX
2017-07-12,O1,European option,2017-07-15,10000.0,Daily,2.9,0.1,GDA_TETSTX


In [942]:
idx_new = list(hourly_c['StartDate'])+list(hourly_c['EndDate'])
time_idx = pd.DataFrame(idx_new,columns=['StartDate']).set_index('StartDate').asfreq(freq='1h')
tmp = time_idx.join(hourly_c.set_index('StartDate'),on='StartDate')
hourly_c = tmp.loc[(tmp.index<idx_new[2])| (tmp.index>=idx_new[1]) & (tmp.index<idx_new[3])].ffill()
hourly_c.index = hourly_c.index.rename('Date')
hourly_c

,ContractName,DealType,EndDate,Volume,Granularity,StrikePrice,Premium,PriceName
Date,,,,,,,,
2017-01-01 00:00:00,S2,Swap,2018-12-31,1000.0,Hourly,21.0,NaN,HB_NORTH
2017-01-01 01:00:00,S2,Swap,2018-12-31,1000.0,Hourly,21.0,NaN,HB_NORTH
2017-01-01 02:00:00,S2,Swap,2018-12-31,1000.0,Hourly,21.0,NaN,HB_NORTH
2017-01-01 03:00:00,S2,Swap,2018-12-31,1000.0,Hourly,21.0,NaN,HB_NORTH
2017-01-01 04:00:00,S2,Swap,2018-12-31,1000.0,Hourly,21.0,NaN,HB_NORTH
...,...,...,...,...,...,...,...,...
2019-09-29 19:00:00,O2,European option,2019-09-30,1000.0,Hourly,31.5,3.0,HB_HOUSTON
2019-09-29 20:00:00,O2,European option,2019-09-30,1000.0,Hourly,31.5,3.0,HB_HOUSTON
2019-09-29 21:00:00,O2,European option,2019-09-30,1000.0,Hourly,31.5,3.0,HB_HOUSTON


## Task 6: Join relevant prices

In [944]:
tmp1 = daily_c.reset_index().set_index(['PriceName','Date'])
tmp2 = fuelP.set_index(['Variable','Date'])
tmp2.index = tmp2.index.rename(['PriceName','Date'])
daily_c_P = tmp1.join(tmp2,on=['PriceName','Date'])
daily_c_P

ContractName         DealType    EndDate   Volume  \
PriceName  Date                                                           
Henry Hub  2017-01-01           S1             Swap 2017-03-31  20000.0   
           2017-01-02           S1             Swap 2017-03-31  20000.0   
           2017-01-03           S1             Swap 2017-03-31  20000.0   
           2017-01-04           S1             Swap 2017-03-31  20000.0   
           2017-01-05           S1             Swap 2017-03-31  20000.0   
...                            ...              ...        ...      ...   
GDA_TETSTX 2017-07-10           O1  European option 2017-07-15  10000.0   
           2017-07-11           O1  European option 2017-07-15  10000.0   
           2017-07-12           O1  European option 2017-07-15  10000.0   
           2017-07-13           O1  European option 2017-07-15  10000.0   
           2017-07-14           O1  European option 2017-07-15  10000.0   

                      Granularity  StrikePrice  Premium  Price  
PriceName  Date                                                 
Henry Hub  2017-01-01       Daily          3.0      NaN  3.650  
           2017-01-02       Daily          3.0      NaN  3.650  
           2017-01-03       Daily          3.0      NaN  3.650  
           2017-01-04       Daily          3.0      NaN  3.390  
           2017-01-05       Daily          3.0      NaN  3.370  
...                           ...          ...      ...    ...  
GDA_TETSTX 2017-07-10       Daily          2.9      0.1  2.865  
           2017-07-11       Daily          2.9      0.1  2.910  
           2017-07-12       Daily          2.9      0.1  2.910  
           2017-07-13       Daily          2.9      0.1  2.915  
           2017-07-14       Daily          2.9      0.1  2.865  

[133 rows x 8 columns]

In [945]:
tmp1 = hourly_c.reset_index().set_index(['PriceName','Date'])
tmp2 = powerP.set_index(['SettlementPoint','Date'])
tmp2.index = tmp2.index.rename(['PriceName','Date'])
hourly_c_P = tmp1.join(tmp2,on=['PriceName','Date'])
hourly_c_P

ContractName         DealType    EndDate  \
PriceName  Date                                                           
HB_NORTH   2017-01-01 00:00:00           S2             Swap 2018-12-31   
           2017-01-01 01:00:00           S2             Swap 2018-12-31   
           2017-01-01 02:00:00           S2             Swap 2018-12-31   
           2017-01-01 03:00:00           S2             Swap 2018-12-31   
           2017-01-01 04:00:00           S2             Swap 2018-12-31   
...                                     ...              ...        ...   
HB_HOUSTON 2019-09-29 19:00:00           O2  European option 2019-09-30   
           2019-09-29 20:00:00           O2  European option 2019-09-30   
           2019-09-29 21:00:00           O2  European option 2019-09-30   
           2019-09-29 22:00:00           O2  European option 2019-09-30   
           2019-09-29 23:00:00           O2  European option 2019-09-30   

                                Volume Granularity  StrikePrice  Premium  \
PriceName  Date                                                            
HB_NORTH   2017-01-01 00:00:00  1000.0      Hourly         21.0      NaN   
           2017-01-01 01:00:00  1000.0      Hourly         21.0      NaN   
           2017-01-01 02:00:00  1000.0      Hourly         21.0      NaN   
           2017-01-01 03:00:00  1000.0      Hourly         21.0      NaN   
           2017-01-01 04:00:00  1000.0      Hourly         21.0      NaN   
...                                ...         ...          ...      ...   
HB_HOUSTON 2019-09-29 19:00:00  1000.0      Hourly         31.5      3.0   
           2019-09-29 20:00:00  1000.0      Hourly         31.5      3.0   
           2019-09-29 21:00:00  1000.0      Hourly         31.5      3.0   
           2019-09-29 22:00:00  1000.0      Hourly         31.5      3.0   
           2019-09-29 23:00:00  1000.0      Hourly         31.5      3.0   

                                Price  
PriceName  Date                        
HB_NORTH   2017-01-01 00:00:00  23.23  
           2017-01-01 01:00:00  21.82  
           2017-01-01 02:00:00  20.16  
           2017-01-01 03:00:00  20.00  
           2017-01-01 04:00:00  20.24  
...                               ...  
HB_HOUSTON 2019-09-29 19:00:00  28.15  
           2019-09-29 20:00:00  23.71  
           2019-09-29 21:00:00  20.95  
           2019-09-29 22:00:00  18.56  
           2019-09-29 23:00:00  17.89  

[24048 rows x 8 columns]

## Task 7: Calculate payoffs

In [946]:
def f(x):
    if x[0][0] == 'S':
        return (x[-1]-x[-3])*x[-5]
    else:
        return (np.max(x[-1]-x[-3])-x[-2])*x[-5]
    
daily_c_P['Payoff'] = daily_c_P.apply(f,axis=1)
hourly_c_P['Payoff'] = hourly_c_P.apply(f,axis=1)

In [947]:
hourly_c_P

ContractName         DealType    EndDate  \
PriceName  Date                                                           
HB_NORTH   2017-01-01 00:00:00           S2             Swap 2018-12-31   
           2017-01-01 01:00:00           S2             Swap 2018-12-31   
           2017-01-01 02:00:00           S2             Swap 2018-12-31   
           2017-01-01 03:00:00           S2             Swap 2018-12-31   
           2017-01-01 04:00:00           S2             Swap 2018-12-31   
...                                     ...              ...        ...   
HB_HOUSTON 2019-09-29 19:00:00           O2  European option 2019-09-30   
           2019-09-29 20:00:00           O2  European option 2019-09-30   
           2019-09-29 21:00:00           O2  European option 2019-09-30   
           2019-09-29 22:00:00           O2  European option 2019-09-30   
           2019-09-29 23:00:00           O2  European option 2019-09-30   

                                Volume Granularity  StrikePrice  Premium  \
PriceName  Date                                                            
HB_NORTH   2017-01-01 00:00:00  1000.0      Hourly         21.0      NaN   
           2017-01-01 01:00:00  1000.0      Hourly         21.0      NaN   
           2017-01-01 02:00:00  1000.0      Hourly         21.0      NaN   
           2017-01-01 03:00:00  1000.0      Hourly         21.0      NaN   
           2017-01-01 04:00:00  1000.0      Hourly         21.0      NaN   
...                                ...         ...          ...      ...   
HB_HOUSTON 2019-09-29 19:00:00  1000.0      Hourly         31.5      3.0   
           2019-09-29 20:00:00  1000.0      Hourly         31.5      3.0   
           2019-09-29 21:00:00  1000.0      Hourly         31.5      3.0   
           2019-09-29 22:00:00  1000.0      Hourly         31.5      3.0   
           2019-09-29 23:00:00  1000.0      Hourly         31.5      3.0   

                                Price   Payoff  
PriceName  Date                                 
HB_NORTH   2017-01-01 00:00:00  23.23   2230.0  
           2017-01-01 01:00:00  21.82    820.0  
           2017-01-01 02:00:00  20.16   -840.0  
           2017-01-01 03:00:00  20.00  -1000.0  
           2017-01-01 04:00:00  20.24   -760.0  
...                               ...      ...  
HB_HOUSTON 2019-09-29 19:00:00  28.15  -6350.0  
           2019-09-29 20:00:00  23.71 -10790.0  
           2019-09-29 21:00:00  20.95 -13550.0  
           2019-09-29 22:00:00  18.56 -15940.0  
           2019-09-29 23:00:00  17.89 -16610.0  

[24048 rows x 9 columns]

## Task 8: Calculate aggregated payoffs

In [948]:
payoff_d = daily_c_P['Payoff'].reset_index().set_index('Date').to_period('M').groupby(['PriceName','Date']).sum()
payoff_h = hourly_c_P['Payoff'].reset_index().set_index('Date').to_period('M').groupby(['PriceName','Date']).sum()

In [949]:
df = pd.concat([payoff_d,payoff_h])
df.columns = ['TotalPayoff']
df.to_csv('./output_files/MonthlyContractPayoffs.csv')

# Plant Dispatch Modeling

## Task 10: Calculate the hourly running cost of each power plant

In [977]:
plantParamenters['Date'] = plantParamenters.apply(lambda x: pd.Timestamp(f'{x[1]}-{x[2]}'),axis=1)
tmp = plantParamenters.set_index(['PlantName','Date'])
tmp

Year  Month PowerPriceName FuelPriceName  Capacity  \
PlantName Date                                                             
Alpha     2017-01-01  2017      1     HB_HOUSTON     Henry Hub       100   
          2017-02-01  2017      2     HB_HOUSTON     Henry Hub        98   
          2017-03-01  2017      3     HB_HOUSTON     Henry Hub        96   
          2017-04-01  2017      4     HB_HOUSTON     Henry Hub        94   
          2017-05-01  2017      5     HB_HOUSTON     Henry Hub        92   
          2017-06-01  2017      6     HB_HOUSTON     Henry Hub        90   
          2017-07-01  2017      7     HB_HOUSTON     Henry Hub        90   
          2017-08-01  2017      8     HB_HOUSTON     Henry Hub        92   
          2017-09-01  2017      9     HB_HOUSTON     Henry Hub        94   
          2017-10-01  2017     10     HB_HOUSTON     Henry Hub        96   
          2017-11-01  2017     11     HB_HOUSTON     Henry Hub        98   
          2017-12-01  2017     12     HB_HOUSTON     Henry Hub       100   
          2018-01-01  2018      1     HB_HOUSTON     Henry Hub       100   
          2018-02-01  2018      2     HB_HOUSTON     Henry Hub        98   
          2018-03-01  2018      3     HB_HOUSTON     Henry Hub        96   
          2018-04-01  2018      4     HB_HOUSTON     Henry Hub        94   
          2018-05-01  2018      5     HB_HOUSTON     Henry Hub        92   
          2018-06-01  2018      6     HB_HOUSTON     Henry Hub        90   
          2018-07-01  2018      7     HB_HOUSTON     Henry Hub        90   
          2018-08-01  2018      8     HB_HOUSTON     Henry Hub        92   
          2018-09-01  2018      9     HB_HOUSTON     Henry Hub        94   
          2018-10-01  2018     10     HB_HOUSTON     Henry Hub        96   
          2018-11-01  2018     11     HB_HOUSTON     Henry Hub        98   
          2018-12-01  2018     12     HB_HOUSTON     Henry Hub       100   
Bravo     2017-01-01  2017      1       HB_SOUTH    GDA_TETSTX       250   
          2017-02-01  2017      2       HB_SOUTH    GDA_TETSTX       245   
          2017-03-01  2017      3       HB_SOUTH    GDA_TETSTX       240   
          2017-04-01  2017      4       HB_SOUTH    GDA_TETSTX       235   
          2017-05-01  2017      5       HB_SOUTH    GDA_TETSTX       230   
          2017-06-01  2017      6       HB_SOUTH    GDA_TETSTX       225   
          2017-07-01  2017      7       HB_SOUTH    GDA_TETSTX       225   
          2017-08-01  2017      8       HB_SOUTH    GDA_TETSTX       230   
          2017-09-01  2017      9       HB_SOUTH    GDA_TETSTX       235   
          2017-10-01  2017     10       HB_SOUTH    GDA_TETSTX       240   
          2017-11-01  2017     11       HB_SOUTH    GDA_TETSTX       245   
          2017-12-01  2017     12       HB_SOUTH    GDA_TETSTX       250   
          2018-01-01  2018      1       HB_SOUTH    GDA_TETSTX       250   
          2018-02-01  2018      2       HB_SOUTH    GDA_TETSTX       245   
          2018-03-01  2018      3       HB_SOUTH    GDA_TETSTX       240   
          2018-04-01  2018      4       HB_SOUTH    GDA_TETSTX       235   
          2018-05-01  2018      5       HB_SOUTH    GDA_TETSTX       230   
          2018-06-01  2018      6       HB_SOUTH    GDA_TETSTX       225   
          2018-07-01  2018      7       HB_SOUTH    GDA_TETSTX       225   
          2018-08-01  2018      8       HB_SOUTH    GDA_TETSTX       230   
          2018-09-01  2018      9       HB_SOUTH    GDA_TETSTX       235   
          2018-10-01  2018     10       HB_SOUTH    GDA_TETSTX       240   
          2018-11-01  2018     11       HB_SOUTH    GDA_TETSTX       245   
          2018-12-01  2018     12       HB_SOUTH    GDA_TETSTX       250   

                      HeatRate  VOM  FuelTransportationCost  FixedStartCost  
PlantName Date                                                               
Alpha     2017-01-01       9.2    3                    0.10      

In [978]:
idx = list(tmp.loc['Alpha'].index)
idx.append(idx[-1]+timedelta(30))
idx = pd.DatetimeIndex(idx,name='Date')

In [979]:
fuelP

,Variable,Date,Price
0,GDA_TETSTX,2019-05-01,2.490
1,GDA_TETSTX,2019-04-30,2.420
2,GDA_TETSTX,2019-04-29,2.445
3,GDA_TETSTX,2019-04-28,2.445
4,GDA_TETSTX,2019-04-27,2.445
...,...,...,...
3677,Henry Hub,2020-09-26,1.895
3678,Henry Hub,2020-09-27,1.895
3679,Henry Hub,2020-09-28,1.895
3680,Henry Hub,2020-09-29,1.835


In [980]:
tmp1 = tmp.loc['Alpha'].reindex(idx).asfreq(freq='1D').ffill().reset_index()
fuelP_tmp = fuelP.set_index(['Variable','Date'])
fuelP_tmp.index = fuelP_tmp.index.rename(['FuelPriceName','Date'])
fuelP_tmp.columns = ['Price_fuel']
tmp1 = tmp1.join(fuelP_tmp,on=['FuelPriceName','Date'])

In [981]:
tmp2 = tmp.loc['Bravo'].reindex(idx).asfreq(freq='1D').ffill().reset_index()
tmp2 = tmp2.join(fuelP_tmp,on=['FuelPriceName','Date'])

In [982]:
tmp1['PlantName'] = 'Alpha'
tmp2['PlantName'] = 'Bravo'

In [983]:
res = pd.concat([tmp1,tmp2])
def running_cost(x):
    return (x[-2]+x[-4])*x[-6] + x[-5]
res['running_cost'] = res.apply(running_cost,axis=1)

In [984]:
res

,Date,Year,Month,PowerPriceName,FuelPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost,Price_fuel,PlantName,running_cost
0,2017-01-01,2017.0,1.0,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.10,10000.0,3.650,Alpha,37.500
1,2017-01-02,2017.0,1.0,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.10,10000.0,3.650,Alpha,37.500
2,2017-01-03,2017.0,1.0,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.10,10000.0,3.650,Alpha,37.500
3,2017-01-04,2017.0,1.0,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.10,10000.0,3.390,Alpha,35.108
4,2017-01-05,2017.0,1.0,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.10,10000.0,3.370,Alpha,34.924
...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,2018-12-27,2018.0,12.0,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0,2.915,Bravo,24.534
726,2018-12-28,2018.0,12.0,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0,3.080,Bravo,25.788
727,2018-12-29,2018.0,12.0,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0,3.080,Bravo,25.788
728,2018-12-30,2018.0,12.0,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0,3.080,Bravo,25.788


## Task 11: Join the hourly power prices

In [985]:
tmp = res.set_index(['PlantName','Date'])
tmp

Year  Month PowerPriceName FuelPriceName  Capacity  \
PlantName Date                                                               
Alpha     2017-01-01  2017.0    1.0     HB_HOUSTON     Henry Hub     100.0   
          2017-01-02  2017.0    1.0     HB_HOUSTON     Henry Hub     100.0   
          2017-01-03  2017.0    1.0     HB_HOUSTON     Henry Hub     100.0   
          2017-01-04  2017.0    1.0     HB_HOUSTON     Henry Hub     100.0   
          2017-01-05  2017.0    1.0     HB_HOUSTON     Henry Hub     100.0   
...                      ...    ...            ...           ...       ...   
Bravo     2018-12-27  2018.0   12.0       HB_SOUTH    GDA_TETSTX     250.0   
          2018-12-28  2018.0   12.0       HB_SOUTH    GDA_TETSTX     250.0   
          2018-12-29  2018.0   12.0       HB_SOUTH    GDA_TETSTX     250.0   
          2018-12-30  2018.0   12.0       HB_SOUTH    GDA_TETSTX     250.0   
          2018-12-31  2018.0   12.0       HB_SOUTH    GDA_TETSTX     250.0   

                      HeatRate  VOM  FuelTransportationCost  FixedStartCost  \
PlantName Date                                                                
Alpha     2017-01-01       9.2  3.0                    0.10         10000.0   
          2017-01-02       9.2  3.0                    0.10         10000.0   
          2017-01-03       9.2  3.0                    0.10         10000.0   
          2017-01-04       9.2  3.0                    0.10         10000.0   
          2017-01-05       9.2  3.0                    0.10         10000.0   
...                        ...  ...                     ...             ...   
Bravo     2018-12-27       7.6  2.0                    0.05         50000.0   
          2018-12-28       7.6  2.0                    0.05         50000.0   
          2018-12-29       7.6  2.0                    0.05         50000.0   
          2018-12-30       7.6  2.0                    0.05         50000.0   
          2018-12-31       7.6  2.0                    0.05         50000.0   

                      Price_fuel  running_cost  
PlantName Date                                  
Alpha     2017-01-01       3.650        37.500  
          2017-01-02       3.650        37.500  
          2017-01-03       3.650        37.500  
          2017-01-04       3.390        35.108  
          2017-01-05       3.370        34.924  
...                          ...           ...  
Bravo     2018-12-27       2.915        24.534  
          2018-12-28       3.080        25.788  
          2018-12-29       3.080        25.788  
          2018-12-30       3.080        25.788  
          2018-12-31       3.080        25.788  

[1460 rows x 11 columns]

In [986]:
tmp1 = tmp.loc['Alpha'].reindex(idx).asfreq(freq='1h').ffill().reset_index()
power_tmp = powerP.set_index(['SettlementPoint','Date'])
power_tmp.index = power_tmp.index.rename(['PowerPriceName','Date'])
power_tmp.columns = ['Power_price']
tmp1 = tmp1.join(power_tmp,on=['PowerPriceName','Date'])

In [987]:
tmp2 = tmp.loc['Bravo'].reindex(idx).asfreq(freq='1h').ffill().reset_index()
tmp2 = tmp2.join(power_tmp,on=['PowerPriceName','Date'])

In [988]:
tmp1.set_index('FuelPriceName','Date').sort_index()

,Date,Year,Month,PowerPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost,Price_fuel,running_cost,Power_price
FuelPriceName,,,,,,,,,,,,
Henry Hub,2017-01-01 00:00:00,2017.0,1.0,HB_HOUSTON,100.0,9.2,3.0,0.1,10000.0,3.650,37.500,23.23
Henry Hub,2017-01-01 01:00:00,2017.0,1.0,HB_HOUSTON,100.0,9.2,3.0,0.1,10000.0,3.650,37.500,21.82
Henry Hub,2017-01-01 02:00:00,2017.0,1.0,HB_HOUSTON,100.0,9.2,3.0,0.1,10000.0,3.650,37.500,20.16
Henry Hub,2017-01-01 03:00:00,2017.0,1.0,HB_HOUSTON,100.0,9.2,3.0,0.1,10000.0,3.650,37.500,20.00
Henry Hub,2017-01-01 04:00:00,2017.0,1.0,HB_HOUSTON,100.0,9.2,3.0,0.1,10000.0,3.650,37.500,20.24
...,...,...,...,...,...,...,...,...,...,...,...,...
Henry Hub,2018-12-30 20:00:00,2018.0,12.0,HB_HOUSTON,100.0,9.2,3.0,0.1,10000.0,4.565,45.918,23.89
Henry Hub,2018-12-30 21:00:00,2018.0,12.0,HB_HOUSTON,100.0,9.2,3.0,0.1,10000.0,4.565,45.918,23.87
Henry Hub,2018-12-30 22:00:00,2018.0,12.0,HB_HOUSTON,100.0,9.2,3.0,0.1,10000.0,4.565,45.918,23.81


In [989]:
res = pd.concat([tmp1,tmp2])

## Task 12: Identify hours in which the power plant should be on.

In [990]:
Generation = lambda x: x[5] if (x[-1]>x[-2]) else 0
res['Generation'] = res.apply(Generation,axis=1)

In [991]:
res['RunningMargin'] = res.apply(lambda x: (x[-2]-x[-3])*x[-1],axis=1)

In [992]:
res.head()

,Date,Year,Month,PowerPriceName,FuelPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost,Price_fuel,running_cost,Power_price,Generation,RunningMargin
0,2017-01-01 00:00:00,2017.0,1.0,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.1,10000.0,3.65,37.5,23.23,0.0,-0.0
1,2017-01-01 01:00:00,2017.0,1.0,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.1,10000.0,3.65,37.5,21.82,0.0,-0.0
2,2017-01-01 02:00:00,2017.0,1.0,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.1,10000.0,3.65,37.5,20.16,0.0,-0.0
3,2017-01-01 03:00:00,2017.0,1.0,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.1,10000.0,3.65,37.5,20.00,0.0,-0.0
4,2017-01-01 04:00:00,2017.0,1.0,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.1,10000.0,3.65,37.5,20.24,0.0,-0.0


## Task 13: Account for start costs

Assume:
- sart costs is 10,000
- a start-shutdown maximume capacity is 12

In [993]:
start_cost = 10000

In [994]:
x = res['RunningMargin'].values

In [1005]:
res_temp = []
for i in range(len(x)):
    if x[i]>0:
        count = 0
        for j in range(i,len(x)):
            if x[j]>0:
                count += x[j]
            else:
                break
        if count-start_cost>0:
            res_temp.append(True)
        else:
            res_temp.append(False)
    else:
        res_temp.append(False)

In [1008]:
res['RunningMargin_satrt_cost'] = np.array(res_temp)*res['RunningMargin']

In [1009]:
res.to_csv('./output_files/PowerPlantDispatch.csv')